In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.3 MB/s eta 0:00:00:00:0100:01


In [2]:
import pickle

In [ ]:
import os

os.environ["HUGGINGFACE_TOKEN"] = ""

In [4]:
from huggingface_hub import login

login(os.environ["HUGGINGFACE_TOKEN"])

In [5]:
import datasets

dataset = "chengxuphd/liar2"
dataset = datasets.load_dataset(dataset)

README.md:   0%|          | 0.00/2.87k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18369 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2297 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [6]:
# Convert labels: {0, 1, 2} -> 0, {3, 4, 5} -> 1
def convert_labels(data):
    if data["label"] in [0, 1, 2]:
        data["label"] = 0
    else:
        data["label"] = 1
    return data

In [7]:
dataset = dataset.map(convert_labels)
test_dataset = dataset["test"]

Map:   0%|          | 0/18369 [00:00<?, ? examples/s]

Map:   0%|          | 0/2297 [00:00<?, ? examples/s]

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [8]:
MODEL_NAME = "meta-llama/few-shot"
CHECKPOINT_PATH = f'{MODEL_NAME}'

In [30]:
def create_directory_if_not_exists(dir_path):
    # Check if the directory exists
    if not os.path.exists(dir_path):
        # Create the directory (and any necessary parent directories)
        os.makedirs(dir_path)
        print(f"Directory '{dir_path}' created.")
    else:
        print(f"Directory '{dir_path}' already exists.")
        
create_directory_if_not_exists(CHECKPOINT_PATH)

Directory 'meta-llama/zero-shot' already exists.


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Use instruction-tuned models

model_name = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
# Zero-shot prompt template
def format_chat_prompt_v1(claim):
    """Formats input using the chat template for better responses (zero-shot)."""
    user_input = f"""Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.
    
    Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
    
    Claim: "{claim}"
    
    Output the result in the following structured format:
    
    Classification: (FAKE or REAL)
    
    Keywords: list of ONLY five important words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
    Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""

    chat_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an AI that detects fake news using linguistic analysis, focusing on tone and word choice, without relying on factual knowledge."},
            {"role": "user", "content": user_input}
        ],
        tokenize=False,  # Keep raw string for LLM
        add_generation_prompt=True,  # Adds assistant's turn automatically
    )
    return chat_prompt

# Few-shot prompt template
def format_chat_prompt_v2(claim):
    """Formats input using the chat template for better responses with few-shot examples."""

    chat_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an AI that classifies a piece of text as FAKE or REAL, using linguistic analysis, focusing on tone and word choice, without relying on factual knowledge."},
            
            # Example 1 (FAKE)
            {"role": "user", "content": """Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.

            Do NOT give any classification other than FAKE or REAL.
            
            Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
            
            Claim: "President Obama used government funds to pay for his personal vacations for the next 20 years and Trump sent him a bill."
            
            Output the result in the following structured format:
    
            Classification: (FAKE or REAL)
    
            Keywords: list of ONLY five important words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
            Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""},
            
            {"role": "assistant", "content": """Classification: FAKE
            
            Keywords: ["government funds", "personal vacations", "20 years", "Trump", "bill"]
            
            Linguistic Reasoning: The claim uses exaggeration, sensational language, and vague accusations (e.g., "20 years", "government funds") to create drama and suspicion without evidence."""},

            # Example 2 (REAL)
            {"role": "user", "content": """Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.
            
            Do NOT give any classification other than FAKE or REAL.
            
            Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
            
            Claim: "Despite having their budget increased by over 40 percent since 2009, pending claims for benefits with the Department of Veterans Affairs have increased from 391,000 to 890,000 under the Obama Administration."
            
            Output the result in the following structured format:
    
            Classification: (FAKE or REAL)
    
            Keywords: list of ONLY five important words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
            Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""},
            
            {"role": "assistant", "content": """Classification: REAL
            
            Keywords: ["budget", "40 percent", "pending claims", "Department of Veterans Affairs", "Obama Administration"]
            
            Linguistic Reasoning: The claim presents specific figures, neutral wording, and clear attribution, making it sound objective and fact-based rather than emotional or exaggerated."""},

            {"role": "user", "content": f"""Classify the following claim as FAKE or REAL, and highlight only the top five keywords contributing to the classification.
            
            Do NOT give any classification other than FAKE or REAL.
            
            Focus only on linguistic features (tone, exaggeration, emotional appeal) and do NOT use any factual knowledge.
            
            Claim: "{claim}"
            
            Output the result in the following structured format:
    
            Classification: (FAKE or REAL)
    
            Keywords: list of ONLY five important words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
            Linguistic Reasoning: (Explain based on language features such as exaggeration, strong emotions, vagueness, etc.)"""}
        ],
        tokenize=False,  # Keep raw string for LLM
        add_generation_prompt=True,  # Adds assistant's turn automatically
    )
    
    return chat_prompt

In [11]:
import re

def extract_prediction(response):
    """Extracts classification, keywords, and reasoning from model output."""
    
    # Extract classification (FAKE or REAL)
    classification_match = re.search(r'Classification:\s*(FAKE|REAL)', response)
    classification = classification_match.group(1) if classification_match else "UNKNOWN"

    # print(response)

    # Extract keywords as a list
    keywords_match = re.search(r'Keywords:\s*\n([\s\S]*)', response)

    if keywords_match:
        keywords_text = keywords_match.group(1).strip()
        
        # Extract keywords handling quoted and unquoted words, and ignoring explanations after `-`
        keywords = re.findall(r'\d+\.\s*["“”]?(.*?)[”"]?(?:\s*-\s*.*)?$', keywords_text, re.MULTILINE)
        keywords = [kw.strip() for kw in keywords]
    else:
        keywords = []

    # Extract linguistic reasoning
    reasoning_match = re.search(r'Linguistic Reasoning:\s*(.*)', response, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."
    
    return classification, keywords, reasoning

In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

true_labels = []
pred_labels = []
results = []
i = 0

In [13]:
with open(f'{CHECKPOINT_PATH}/true_labels.pkl', 'rb') as file:
    true_labels = pickle.load(file)

with open(f'{CHECKPOINT_PATH}/pred_labels.pkl', 'rb') as file:
    pred_labels = pickle.load(file)

with open(f'{CHECKPOINT_PATH}/results.pkl', 'rb') as file:
    results = pickle.load(file)

i = len(true_labels)
print(i)

1899


In [14]:
for sample in test_dataset.select(range(i, len(test_dataset))):
    if i % 100 == 0:
        print(f'{i} done')
        with open(f'{CHECKPOINT_PATH}/true_labels.pkl', 'wb') as file:
            pickle.dump(true_labels, file)
        
        with open(f'{CHECKPOINT_PATH}/pred_labels.pkl', 'wb') as file:
            pickle.dump(pred_labels, file)

        with open(f'{CHECKPOINT_PATH}/results.pkl', 'wb') as file:
            pickle.dump(results, file)

                
    i += 1
    
    claim = sample["statement"]
    true_label = sample["label"]

    # Format prompt using chat template
    formatted_prompt = format_chat_prompt_v2(claim)

    # Tokenize and generate response
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
    output_tokens = model.generate(**inputs, max_new_tokens=150)
    model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    cleaned_response = model_response.split("[/INST]")[-1].strip()

    # Extract structured outputs
    predicted_label, keywords, reasoning = extract_prediction(cleaned_response)
    # print(keywords)

    # Convert predicted label to binary (FAKE=0, REAL=1)
    if predicted_label  == "UNKNOWN":
        continue
    pred_label = 0 if predicted_label == "FAKE" else 1

    # Store results
    true_labels.append(true_label)
    pred_labels.append(pred_label)

    # # Print one sample response for debugging
    # print(f"Claim: {claim}")
    # print(f"Prediction: {predicted_label}")
    # print(f"Keywords: {keywords}")
    # print(f"Linguistic Reasoning: {reasoning}\n")
    
    results.append({
            "Claim": claim,
            "Prediction": predicted_label,
            "Keywords": ", ".join(keywords),
            "Linguistic Reasoning": reasoning,
            "True Label": true_label
        })

df = pd.DataFrame(results)
df.to_csv(f"{CHECKPOINT_PATH}/results.csv", index=False)


1900 done
2000 done
2100 done
2200 done


In [15]:
import pickle

with open(f'{CHECKPOINT_PATH}/true_labels.pkl', 'wb') as file:
    pickle.dump(true_labels, file)

with open(f'{CHECKPOINT_PATH}/pred_labels.pkl', 'wb') as file:
    pickle.dump(pred_labels, file)

In [16]:
# Compute evaluation metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average="binary")

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.6328
Precision: 0.6868
Recall: 0.2456
F1-score: 0.3618


**Llama**
- Zero shot:
```
Accuracy: 0.4673
Precision: 0.4420
Recall: 0.9794
F1-score: 0.6091
```

- Few shot:
```
Accuracy: 0.6328
Precision: 0.6868
Recall: 0.2456
F1-score: 0.3618
```

## Debugging Purposes

In [ ]:
# for i in range(10):
#     train_claim = dataset["train"]["statement"][i]
#     train_label = dataset["train"]["label"][i]
#     print(f"Claim:\n{train_claim}")
#     formatted_prompt = format_chat_prompt(train_claim)
    
#     inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
#     output_tokens = model.generate(**inputs, max_new_tokens=150)
#     model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
#     cleaned_response = model_response.split("[/INST]")[-1].strip()
#     print(f"Model Response:\n{cleaned_response}")
#     print(f"True Label:\n{train_label}")

#     print(extract_prediction(cleaned_response))
#     print("\n#########################")